In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

from tensorflow.keras.optimizers import SGD

from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from metrics import MCC_binary, MCC_multi

import Models

Using TensorFlow backend.


In [2]:
# Enable FP16

# https://medium.com/@noel_kennedy/how-to-use-half-precision-float16-when-training-on-rtx-cards-with-tensorflow-keras-d4033d59f9e4

import tensorflow.keras.backend as K

K.set_floatx('float16')

# default is 1e-7 which is too small for float16.  
# Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
K.set_epsilon(1e-4)

In [3]:
# DATA PATHS

TRAIN_FOLDER_PATH = './data/train/'
VAL_FOLDER_PATH = './data/validation/'
TEST_FOLDER_PATH = './data/test/'

In [4]:
# Hyper parameters

learning_rate = 2e-5
batch_size = 20
num_epochs = 40
classes = 2
loss = "categorical_crossentropy"

In [5]:
# Image properties

img_width = 299
img_height = 299
img_depth = 3

In [6]:
# Data generator

data_gen = ImageDataGenerator(
    rescale = 1 / 255.0,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip = True,
    fill_mode = "nearest"
)

def get_train_generator(img_height, img_width):
    train_generator = data_gen.flow_from_directory(
        TRAIN_FOLDER_PATH,
        target_size = (img_height, img_width),
        color_mode = 'rgb',
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True
    )
    
    return train_generator

def get_validation_generator(img_height, img_width):
    validation_generator = data_gen.flow_from_directory(
        VAL_FOLDER_PATH,
        target_size = (img_height, img_width),
        color_mode = 'rgb',
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = True
    )
    
    return validation_generator

def get_test_generator(img_height, img_width):
    test_generator = data_gen.flow_from_directory(
        TEST_FOLDER_PATH,
        target_size = (img_height, img_width),
        color_mode='rgb',
        batch_size = batch_size,
        class_mode = 'categorical',
        shuffle = False
    )
    
    return test_generator

In [7]:
# Callbacks
def get_callbacks():
    mc = ModelCheckpoint('best_fp16.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
    
    return [mc]


In [8]:
# Compile the model

def compile_model(model):
    opt = SGD(lr = learning_rate)
    
    model.compile(loss = loss, optimizer = opt, metrics=["accuracy"])
    
    return model

In [9]:
# Fit the model

def fit_model(model, train_generator, validation_generator):
    history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.n // train_generator.batch_size,
        epochs = num_epochs,
        validation_data = validation_generator,
        validation_steps = validation_generator.n // validation_generator.batch_size,
        use_multiprocessing = False,
        callbacks = get_callbacks())
    
    return history, model


In [10]:
# Save model
def save_model(model, model_name):
    model.save(model_name + '.h5')

In [11]:
# Evaluate the model

def evaluate_model(model, validation_generator):
    return model.evaluate_generator(validation_generator, steps = validation_generator.n // validation_generator.batch_size)


In [12]:
# Predictions

def get_predictions(model, test_generator):
    test_generator.reset()
    
    predIdx = model.predict_generator(test_generator, steps = (test_generator.n // test_generator.batch_size) + 1)
    
    predIdx = np.argmax(predIdx, axis=1)
    
    return predIdx

In [13]:
# Plot accuracy and loss

def plot_metrics(history):
    plt.figure(figsize=(15,10))
    plt.subplot(2,2,1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='lower right')

    # summarize history for loss
    plt.subplot(2,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    
    # Plot MCC_binary values
    #plt.subplot(2,2,3)
    #plt.plot(history.history['MCC_binary'])
    #plt.plot(history.history['val_MCC_binary'])
    #plt.title('model MCC_binary')
    #plt.ylabel('MCC_binary')
    #plt.xlabel('epoch')
    #plt.legend(['train', 'validation'], loc='lower right')
    
    plt.show()

In [14]:
# Show evaluation metrics

def show_eval_metrics(model, metrics):
    for idx, m in enumerate(model.metrics_names):
        print("{}: {:.4f}".format(m, metrics[idx]))


In [15]:
# Show test metrics

def show_test_metrics(predictions, test_generator):
    print("## Classification report ##")
    print(classification_report(test_generator.classes, predictions, target_names=test_generator.class_indices.keys()))

    cm = confusion_matrix(test_generator.classes, predictions)
    
    total = sum(sum(cm))
    acc = (cm[0,0] + cm[1,1]) / total
    sensitivity = cm[0,0] / (cm[0,0] + cm[0,1])
    specificity = cm[1,1] / (cm[1,0] + cm[1,1])

    print("## Other values ##")
    print("acc: {:.4f}".format(acc))
    print("sensitivity: {:.4f}".format(sensitivity))
    print("specificity: {:.4f}".format(specificity))
    print("MCC multi: {:.4f}".format(MCC_multi(cm)))
    

In [16]:
# Plot confusion matrix

def plot_confusion_matrix(predictions, test_generator, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = confusion_matrix(test_generator.classes, predictions)
    classes = test_generator.class_indices.keys()
    
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [17]:
# Create the generators

train_generator = get_train_generator(img_height, img_width)
validation_generator = get_validation_generator(img_height, img_width)
test_generator = get_test_generator(img_height, img_width)

Found 227 images belonging to 2 classes.
Found 73 images belonging to 2 classes.
Found 47 images belonging to 2 classes.


In [ ]:
# Xception model

from tensorflow.keras.applications.xception import Xception

xception_model = Xception(weights='imagenet', include_top=False, input_shape = (img_height, img_width, img_depth))

model = Sequential()
model.add(xception_model)

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Dense(2))
model.add(Activation("softmax"))

xception_model.trainable = False

In [20]:
# EfficientNet model
img_width = 224
img_height = 224
model = Models.build_efficientnet(img_height, img_width, img_depth)
model.summary()

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
# Compile

model = compile_model(model)

In [ ]:
# Train

history, model = fit_model(model, train_generator, validation_generator)

In [ ]:
# Save model
save_model(model, 'fp16')

In [ ]:
# Evaluate

metrics = evaluate_model(model, validation_generator)
show_eval_metrics(model, metrics)

In [ ]:
# Plot metrics from history

plot_metrics(history)

In [ ]:
# Load model

from tensorflow.keras.models import load_model

model = load_model('fp16.h5', custom_objects={'MCC_binary':MCC_binary})

In [ ]:
# Predictions

predictions = get_predictions(model, test_generator)

In [ ]:
# Confusion matrix

plot_confusion_matrix(predictions, test_generator)

In [ ]:
# Show final metrics

show_test_metrics(predictions, test_generator)